<a href="https://colab.research.google.com/github/spoupry/AQdata/blob/main/AggregatorV5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modules


In [30]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from statsmodels.nonparametric.kernel_regression import KernelReg
from numpy import linspace
from datetime import datetime, timedelta

# Importation données locales

In [31]:
# Import data from GitHub
url1 = 'https://github.com/spoupry/AQdata/raw/main/Mesures_ARG_dec2021-juin2022.csv'
# Read and use csv file Local measures from Argeles-gazost
df = pd.read_csv(url1, delimiter=";")
print(df.head(1))
# df.dtypes
# Format et selection datas (PM10)
# Converti le format utc en format de date local (France)
df['time'] = pd.to_datetime(df['time']).dt.tz_convert("Europe/Paris")
# Met en format naive timestamp tout en conservant l'UTC
df['time'] = df['time'].dt.tz_localize(None)
# Réduit la précision des dates à la seconde
df['time'] = df['time'].values.astype('datetime64[s]')
print(df.time.head(2))
# selection des colonnes
data_select = df[["time", "Id_Smart_Capteur", "Humidite", "Pression", "PM10", "PM25", "Temperature"]]
# extraction des données par une copie
local_df = data_select.copy()
# Setting the timestamp as index of the dataframe. THis helps to iterate through the data based on time
local_df.set_index('time', inplace=True)
print(local_df.head())
local_df.dtypes
# Données argelès local sont prêtes !!!

                       time  CH4        CO  CO_pas  Humidite Id_Smart_Capteur  \
0  2021-12-08T14:13:25.636Z  0.0  0.116999     NaN  37.36719         Arg12b1f   

   MQ4_pas  NH3  NH3_pas       NO2  ...  SO2  SO2_Vgas  SO2_temperature  TVOC  \
0      NaN  0.0      NaN  0.473526  ...  0.0       NaN              NaN     0   

   Temperature  Timestamp         X         Y     Z  eCO2  
0        19.13          0  43.23295  0.078082  10.3   400  

[1 rows x 24 columns]
0   2021-12-08 15:13:25
1   2021-12-08 15:18:27
Name: time, dtype: datetime64[ns]
                    Id_Smart_Capteur  Humidite  Pression  PM10  PM25  \
time                                                                   
2021-12-08 15:13:25         Arg12b1f  37.36719  954.4344   6.2   2.0   
2021-12-08 15:18:27         Arg12b1f  29.59570  954.4305   6.8   2.2   
2021-12-08 15:23:24         Arg12b1f  27.83105  954.3577   5.0   2.4   
2021-12-08 15:27:35         Arg12b2f  33.38086  955.0228   9.9   2.3   
2021-12-08 16:22:

Id_Smart_Capteur     object
Humidite            float64
Pression            float64
PM10                float64
PM25                float64
Temperature         float64
dtype: object

# Traitements spécial (erreur d'ID)

In [32]:
# Recherche des capteurs de la station locale
uniq_vals = local_df['Id_Smart_Capteur'].unique()
print(uniq_vals)
# extraction des capteurs et de leur mesures
Arg12b1f = local_df[local_df['Id_Smart_Capteur'] == 'Arg12b1f']
Arg12b2f = local_df[local_df['Id_Smart_Capteur'] == 'Arg12b2f']
Arg12b3f = local_df[local_df['Id_Smart_Capteur'] == 'Arg12b3f']
ENI0122B1V1 = local_df[local_df['Id_Smart_Capteur'] == 'ENI0122B1V1']  # Arg12b2f 
ENI0122B2V1 = local_df[local_df['Id_Smart_Capteur'] == 'ENI0122B2V1']  # Arg12b3f
# Réattribution des noms des capteurs en fonction de leur place dans la station de mesure
# concat Arg12b2f avec ENI0122B1V1
Arg12b2f = pd.concat([Arg12b2f,ENI0122B1V1])
Arg12b2f.sort_index(inplace=True)
# recherche des valeurs autres que Arg12b..
print(Arg12b2f['Id_Smart_Capteur'].unique())
Arg12b2f = Arg12b2f.replace('ENI0122B1V1','Arg12b2f')
# concat Arg12b3f avec ENI0122B2V1
Arg12b3f = pd.concat([Arg12b3f,ENI0122B2V1])
Arg12b3f.sort_index(inplace=True)
# recherche des valeurs autres que Arg12b..
print(Arg12b3f['Id_Smart_Capteur'].unique())
Arg12b3f = Arg12b3f.replace('ENI0122B2V1','Arg12b3f')
# Confirmation des remplacements
print("Confirmation des IDs uniques")
print(Arg12b2f['Id_Smart_Capteur'].unique())
print(Arg12b3f['Id_Smart_Capteur'].unique())
#création dataframe d'entrée (celui qui est censé avoir de base)
dfin = pd.concat([Arg12b1f,Arg12b2f,Arg12b3f])
dfin.sort_index(inplace=True)
print("Confirmation des IDs uniques")
print(dfin['Id_Smart_Capteur'].unique())
# création petit dataframme pour alléger calculs
df100 = dfin[14000:14300]
df100

['Arg12b1f' 'Arg12b2f' 'ENI0122B1V1' 'ENI0122B2V1' 'Arg12b3f']
['Arg12b2f' 'ENI0122B1V1']
['ENI0122B2V1' 'Arg12b3f']
Confirmation des IDs uniques
['Arg12b2f']
['Arg12b3f']
Confirmation des IDs uniques
['Arg12b1f' 'Arg12b2f' 'Arg12b3f']


,Id_Smart_Capteur,Humidite,Pression,PM10,PM25,Temperature
time,,,,,,
2022-02-04 07:47:14,Arg12b3f,44.33398,968.2578,9.5,5.1,12.90
2022-02-04 07:47:26,Arg12b2f,41.98828,968.4969,7.6,6.7,12.63
2022-02-04 07:50:48,Arg12b1f,45.15430,968.5073,8.2,5.5,12.38
2022-02-04 07:52:17,Arg12b3f,44.15234,968.3089,10.4,4.9,13.02
2022-02-04 07:52:30,Arg12b2f,41.54980,968.4961,6.5,5.9,12.91
...,...,...,...,...,...,...
2022-02-04 19:58:08,Arg12b3f,41.29199,969.9221,7.7,4.4,19.80
2022-02-04 19:58:19,Arg12b2f,39.23145,969.9442,6.1,5.6,19.65
2022-02-04 20:03:11,Arg12b3f,41.18555,969.9666,7.7,3.7,19.77


# Fonctions annexes

In [33]:
# Calculate the median of a list
# Input : time-serie
# Output the median as a float or None if list =[]
def calculate_median(l):
    l = sorted(l)
    l_len = len(l)
    if l_len < 1:
        return None
    if l_len % 2 == 0 :
        return ( l[(l_len)//2] + l[(l_len//2)-1] ) / 2.0
    else:
        return l[(l_len-1)//2]

In [34]:
# Renvoie l'élément manquant par comparaison des deux listes
# IN: liste de référence, liste à contrôler
# Out: une liste des élements manquants
def missing_element(reference,liste):
  set1 = set(reference)
  set2 = set(liste)
  missing = list(sorted(set1 - set2))
  # print(missing)
  # pour extra elements
  # added = list(sorted(set2 - set1))
  # print(added)
  return missing
# Test de fonctionnement
# ref = ['Arg12b3f', 'Arg12b2f', 'Arg12b1f']
# detec  = ['Arg12b2f', 'Arg12b1f']
# missing = ['Arg12b3f']
# missing_element(ref,detec)

In [35]:
# Détection défaillant capteur bloqué à une valeur
# In: dataframe avec deux colonnes ('Id_Smart_Capteur + parametre')
# OUT: dataframe avec colonne supplémentaire ('parametref') 
def recherche_defaillance(df,parametre,liste_SmS):
  # Extraction des mesures des trois capteurs de la liste pour rechercher des défaillance
  # print(df)
  values_param = []
  block = []
  for i in liste_SmS:
    yp = df[df['Id_Smart_Capteur'] == i][parametre].to_list()
    if(len(set(yp))==1):
      # print("Tous les éléments de la liste sont identiques",yp)
      block.append(i)
    else:
      # print("Tous les éléments de la liste ne sont pas identiques",yp)
      values_param = values_param + yp
  # print(block)
  return values_param , block


## filtration + kernel regression (inutile)

In [36]:
# Smoothing curve by apply kernel regression
# Input: time serie of reliable values
# Output time serie filtrated
def kernel_regression(X1):
  # Extract times series 

  # print("Start filtering")
  # Must have two points minimum
  if(len(X1) == 0 or len(X1) == 1):
    X1f = X1
    # print("not enough data",X1)
  else:
    X1_ = X1.copy()
    X1_.index = linspace(1., len(X1_),len(X1_))
    kr = KernelReg([X1_.values],[X1_.index.values], var_type='u')
    f1 = kr.fit([X1_.index.values])
    X1f = pd.Series(data=f1[0], index = X1.index, dtype=float)
  # print("End filtering")
  # Return filtered data time-series
  return X1f


In [37]:
# Filtration: Use Kernel regression on parameter
# In: dataframe avec deux colonnes ('Id_Smart_Capteur + parametre')
# OUT: dataframe avec colonne supplémentaire ('parametref') 
def filtration(df):
  # init variable de sortie
  filtered = df.iloc[:0,:].copy()
  # recupération de la liste des capteurs en service
  SmS_liste = df['Id_Smart_Capteur'].unique()
  # print("capteurs:",SmS_liste)
  # recupération du parametre à lisser
  col = df.columns
  # print("parametre:",col[1])
  parametre = col[1]
  # print("tranche à filter",df)
  for capteur in SmS_liste:
    X = df[df['Id_Smart_Capteur'] == capteur].copy()
    # print(capteur,X)
    X[parametre +'filtered'] = kernel_regression(X[parametre]).values
    # print("filtered",X)
    filtered = pd.concat([filtered,X])
    # print("tranche filtrée",filtered)
    filtered.sort_index(inplace=True)
  return filtered

## derivée liste

In [38]:
# calcul dérivé d'une liste
l = [2.4, 0.0, 0.0, 3.4, 5.7, 0.0, 3.8, 0.0, 0.0, 4.4, 0.0, 0.0, 2.3, 0.0, 0.0]
l1 = [10.0]*5

print(l[1:])
print(l[:-1])
# la soustraction des listes ne peut pas se faire directement
yp = np.array(l[1:]) - np.array(l[:-1])
print(yp)
if(len(set(yp))==1):
    print("Tous les éléments de la liste sont identiques")
else:
    print("Tous les éléments de la liste ne sont pas identiques")


[0.0, 0.0, 3.4, 5.7, 0.0, 3.8, 0.0, 0.0, 4.4, 0.0, 0.0, 2.3, 0.0, 0.0]
[2.4, 0.0, 0.0, 3.4, 5.7, 0.0, 3.8, 0.0, 0.0, 4.4, 0.0, 0.0, 2.3, 0.0]
[-2.4  0.   3.4  2.3 -5.7  3.8 -3.8  0.   4.4 -4.4  0.   2.3 -2.3  0. ]
Tous les éléments de la liste ne sont pas identiques


# Fontion de synthèse

In [39]:
# In: Dataframe, threshold seuil de tolérance, tranche de temps (Tau) 
# In: parametre, liste des nom des capteurs de la station
# OUT: Synthesis of 
def parametre_synthesis(df,threshold,tau,parametre,ref_SmS):
  # df(pd.dataframe,int,int,string,list of string)
  print("polluant traité",parametre)
  # extraction df pour le parametre choisit
  df_parametre = df[['Id_Smart_Capteur', parametre]]
  # print(df_parametre)
  print("taille du dataframe", len(df_parametre.index))
  # convertion int en string et récupération de la moitié pour décaler la tranche
  tau_demi = round(tau/2)
  tau = str(tau) + 'min'
  # tau_demi = str(tau_demi) + 'min'
  print("Tranches de ",tau,"et décalage de ",tau_demi,"min")
  # 1 Decoupe le dataframe en tranches de temps
  t = tuple(df_parametre.resample(tau))
  print("Nb de tranches:",len(t))
  print("StartTime df",t[0][0]," | EndTime df",t[-1][0]  )
  # print(t)
  # Création des listes pour dataframe de sortie
  sliceTime = []
  nb_sensor = []
  detect = []
  median = []
  median_voter = []
  outlier_slice = pd.DataFrame()
  reliable_slice = pd.DataFrame()
  filtered_slice = pd.DataFrame()
  reliable = df_parametre.iloc[:0,:].copy()
  outlier = df_parametre.iloc[:0,:].copy()
  # Pour chaque tranche de temps on traite les dataframes
  # Ici row fait référence à un timestamp row[0] + tranche df row[1]
  for row in t:
    # Si pas de capteur alors dataframe vide, on regarde l'élément suivant
    if row[1].empty:
      print("dataframe vide at ",row[0])
      continue

    # print("time",row[0])
    # print("time+1/2",row[0] + timedelta(minutes = tau_demi))
    #  tranche de date (ajout de la demi tranche pour center)
    sliceTime.append(row[0] + timedelta(minutes = tau_demi))
    # Nombre de capteur en service
    liste_SmS = row[1]['Id_Smart_Capteur'].unique()
    nb_sensor.append(len(liste_SmS))
    # Identification capteur manquant
    Fault_SmS = missing_element(ref_SmS,liste_SmS)
    
    
    # Recupération des valeurs des capteurs du paramètre
    # recherche et exclusion des capteurs bloqués à une valeur
    r = recherche_defaillance(row[1],parametre,liste_SmS)
    # print(r)
    values_param = r[0]
    block = r[1]
    Fault_SmS = Fault_SmS + block
    Fault_SmS.append("B")
    detect.append(Fault_SmS)
    
    # values_param = row[1][parametre].to_list()
    # print("liste des valeurs des paramêtres pour calculer la médiane")
    # print(values_param)
    # Calcul de la médiane des valeurs récoltées
    median_slice = calculate_median(values_param)
    median.append(median_slice)
    
    # Detection outliers a partir de seuils mobiles defini par threshold
    medmin = median_slice - (median_slice*threshold)/100
    medmax = median_slice + (median_slice*threshold)/100
    outlier_slice = row[1][(row[1][parametre] > medmax) | (row[1][parametre] < medmin)]
    # Recupération des valeurs dans les seuils pour analyse ultérieure
    # reliable_slice = pd.concat([row[1], outlier_slice, outlier_slice]).drop_duplicates(keep=False)
    reliable_slice = row[1][(row[1][parametre] <= medmax) & (row[1][parametre] >= medmin)]
    # Test: len row = len out + len rel
    # print("row[1]:", len(row[1].index) ," out:", len(outlier_slice.index), " rel:", len(reliable_slice.index) )
    
    # filtered slice
    # desactive pour gain de temps
    # filtered_slice = filtration(reliable_slice)
    # ne fait rien mais evite la regression (pas vraiment utile en fait)
    filtered_slice = reliable_slice
    # median voter
    voter_slice = calculate_median(filtered_slice[parametre].to_list())
    median_voter.append(voter_slice)

    # Store value in DataFrame
    outlier = pd.concat([outlier,outlier_slice])
    reliable = pd.concat([reliable,filtered_slice])

  synthese = pd.DataFrame(list(zip(nb_sensor, detect, median, median_voter)), index = sliceTime, columns=['Nb','Fault', 'median' + parametre, 'voteur' + parametre])
  print("tailles des dataframes:","df:",len(df_parametre.index),"synthèse:",len(synthese.index),"ouliers:",len(outlier.index),"reliable:", len(reliable.index))
  return synthese,outlier,reliable

# Fonction affichage

In [59]:

# Affiche les resultats de l'algorithme
def affiche_result(synthese,outlier,reliable,threshold,tau,parametre,ref_SmS ):
  # Préparation des données
  # medmin = median_slice - (median_slice*threshold)/100
  # medmax = median_slice + (median_slice*threshold)/100
  y_upper = synthese['median' + parametre] + (threshold * synthese['median' + parametre])/100
  y_lower = synthese['median' + parametre] - (threshold * synthese['median' + parametre])/100
  # raw data
  X1out = outlier[outlier['Id_Smart_Capteur'] == ref_SmS[0]]
  X2out = outlier[outlier['Id_Smart_Capteur'] == ref_SmS[1]]
  X3out = outlier[outlier['Id_Smart_Capteur'] == ref_SmS[2]]
  E1 = len(X1out.index)
  print(E1)

  X1rel = reliable[reliable['Id_Smart_Capteur'] == ref_SmS[0]]
  X2rel = reliable[reliable['Id_Smart_Capteur'] == ref_SmS[1]]
  X3rel = reliable[reliable['Id_Smart_Capteur'] == ref_SmS[2]]
  R1 = len(X1rel.index)

  import plotly.graph_objects as go
  import plotly.express as px
  # from datetime import datetime, timedelta
  # median_index = synthese.index + timedelta(minutes = 30)

   
  fig = go.Figure()


    # Raw plots
  X1raw = pd.concat([X1out,X1rel])
  X2raw = pd.concat([X2out,X2rel])
  X3raw = pd.concat([X3out,X3rel])
  X1raw.sort_index(inplace=True)
  X2raw.sort_index(inplace=True)
  X3raw.sort_index(inplace=True)
  fig.add_trace(go.Scatter(
    x=X1raw.index,
    y=X1raw[parametre],
    line=dict(width=1, color='blue'),
    name=ref_SmS[0] + 'raw',
  ))

  fig.add_trace(go.Scatter(
    x=X2raw.index,
    y=X2raw[parametre],
    line=dict(width=1, color='orange'),
    name=ref_SmS[1] + 'raw',
  ))
    
  fig.add_trace(go.Scatter(
    x=X3raw.index,
    y=X3raw[parametre],
    line=dict(width=1, color='brown'),
    name=ref_SmS[2] + 'raw',
  ))     


  # Ouliers plots
  fig.add_trace(go.Scatter(
    x=X1out.index,
    y=X1out[parametre],
    mode="markers",
    name=ref_SmS[0] + 'out',
    marker = dict(size = 4, color = 'red', symbol = 'circle'),
    
  ))

  fig.add_trace(go.Scatter(
    x=X2out.index,
    y=X2out[parametre],
    mode="markers",
    name=ref_SmS[1] + 'out',
    marker = dict(size = 4, color = 'red', symbol = 'x'),
  ))
    
  fig.add_trace(go.Scatter(
    x=X3out.index,
    y=X3out[parametre],
    mode="markers",
    name=ref_SmS[2] + 'out',
    marker = dict(size = 4, color = 'red', symbol = 'cross'),
  ))  

  # # Reliable plots
  # fig.add_trace(go.Scatter(
  #   x=X1rel.index,
  #   y=X1rel[parametre],
  #   name=ref_SmS[0] + 'rel',
  # ))

  # fig.add_trace(go.Scatter(
  #   x=X2rel.index,
  #   y=X2rel[parametre],
  #   name=ref_SmS[1] + 'rel',
  # ))
    
  # fig.add_trace(go.Scatter(
  #   x=X3rel.index,
  #   y=X3rel[parametre],
  #   name=ref_SmS[2] + 'rel',
  # ))     
 # Synthèse
  fig.add_trace(go.Scatter(
    x=synthese.index,
    y=synthese['Nb'],
    name='Nombre SmS',
  ))

  fig.add_trace(go.Scatter(
    x=synthese.index,
    y=synthese['median' + parametre],
    line=dict(width=1, color='black'),
    name='Median' + parametre,
  ))
  fig.add_trace(go.Scatter(
    x=np.concatenate([synthese.index, synthese.index[::-1]]),
    y=pd.concat([y_upper, y_lower[::-1]]),
    fill='toself',
    line=dict(width=0.5, color='rgb(150, 247, 212)'),
    # fillcolor='rgb(184, 247, 212)',
    hoveron='points',
    name='Medianbonds',
    # mode='none'
  ))

  # fig.add_trace(go.Scatter(
  #   x=synthese.index,
  #   y=synthese['voteur' + parametre],
  #   line=dict(width=1, color='black'),
  #   name='voteur' + parametre,
  # ))

  # permet de rajouter la visualisation dynamique issue des stockcharts
  # fig.update_xaxes(
  #   rangeslider_visible=True,

  # )
  # Change grid color and axis colors
  # fig.update_xaxes(showline=True, linewidth=0.5, linecolor='grey', gridcolor='grey')
  fig.update_yaxes(showline=True, linewidth=0.5, linecolor='grey', gridcolor='grey')
  fig.update_layout(title_text='Synthese', title_x=0.5, plot_bgcolor='rgba(0,0,0,0)')
  # add annotation
  fig.add_annotation(dict(font=dict(color='black',size=10),
  x=1,
  y=1,
  showarrow=False,
  text=f"{ref_SmS[0]} out: {len(X1out.index)} / {len(X1out.index) + len(X1rel.index)} <br>{ref_SmS[1]} out: {len(X1out.index)} / {len(X2out.index) + len(X2rel.index)} <br>{ref_SmS[2]} out: {len(X3out.index)} / {len(X3out.index) + len(X3rel.index)} ",
  textangle=0,
  xanchor='right',
  xref="paper",
  yref="paper"))
  fig.show()


# Utilisation Algorithme

In [ ]:
data = dfin.loc['2022-04-09':'2022-04-16']
print(data)


                    Id_Smart_Capteur  Humidite  Pression  PM10  PM25  \
time                                                                   
2022-04-09 00:02:49         Arg12b3f  46.67090  962.9598   2.4   1.4   
2022-04-09 00:04:07         Arg12b2f  48.49414  962.7517   0.0   0.0   
2022-04-09 00:04:20         Arg12b1f  46.51465  962.7866   0.0   0.0   
2022-04-09 00:14:55         Arg12b3f  45.75391  963.1461   3.4   1.3   
2022-04-09 00:16:13         Arg12b2f  48.74219  962.9116   5.7   1.7   
...                              ...       ...       ...   ...   ...   
2022-04-16 23:41:04         Arg12b2f  39.71875  969.2076   2.1   1.6   
2022-04-16 23:43:55         Arg12b1f  37.58594  969.1855   0.0   0.0   
2022-04-16 23:50:04         Arg12b3f  38.40820  969.3220   3.4   2.5   
2022-04-16 23:53:09         Arg12b2f  40.17676  969.1312   2.1   1.6   
2022-04-16 23:56:01         Arg12b1f  38.06738  969.1186   0.0   0.0   

                     Temperature  
time                        

In [60]:
# data = dfin[14000:14300]
# data = dfin.loc['2021-12-08':'2021-12-31']
data = dfin.loc['2022-04-09':'2022-04-16']
# data = dfin.loc['2022-04-09 00:00:00':'2022-04-09 02:00:00']
# seuil de tolérance pour le capteur 
threshold = 35
# trache de temps en minutes
tau = 60
# paramètre choisit
parametre = 'PM10'
# liste des noms des SmS affiliés à la station
ref_SmS = ['Arg12b1f', 'Arg12b2f', 'Arg12b3f']
# Utilisation de la fonction de synthèse + outlier + reliable
result = parametre_synthesis(data,threshold,tau,parametre,ref_SmS)
synthese = result[0]
outlier = result[1]
reliable = result[2]
print("synthèse")
print(synthese.head(),"\n")
print("outliers")
print(outlier.head(),"\n")
print("reliable")
print(reliable.head(),"\n")

# affichage
affiche_result(synthese,outlier,reliable,threshold,tau,parametre,ref_SmS )


polluant traité PM10
taille du dataframe 2861
Tranches de  60min et décalage de  30 min
Nb de tranches: 192
StartTime df 2022-04-09 00:00:00  | EndTime df 2022-04-16 23:00:00
tailles des dataframes: df: 2861 synthèse: 192 ouliers: 1602 reliable: 1259
synthèse
                     Nb                    Fault  medianPM10  voteurPM10
2022-04-09 00:30:00   3            [Arg12b1f, B]        2.35        2.35
2022-04-09 01:30:00   3  [Arg12b2f, Arg12b1f, B]        2.80        2.75
2022-04-09 02:30:00   3            [Arg12b1f, B]        3.20        3.40
2022-04-09 03:30:00   3  [Arg12b2f, Arg12b1f, B]        4.00        4.00
2022-04-09 04:30:00   3            [Arg12b1f, B]        6.90        7.10 

outliers
                    Id_Smart_Capteur  PM10
time                                      
2022-04-09 00:04:07         Arg12b2f   0.0
2022-04-09 00:04:20         Arg12b1f   0.0
2022-04-09 00:14:55         Arg12b3f   3.4
2022-04-09 00:16:13         Arg12b2f   5.7
2022-04-09 00:16:25         Arg12

## Playground

In [ ]:
#test fonction
threshold = 20
tau = 60
parametre = 'PM10'
ref_SmS = ['Arg12b1f', 'Arg12b2f', 'Arg12b3f']
affiche_result(synthese,outlier,reliable,threshold,tau,parametre,ref_SmS )

954


# Essais

In [ ]:
import plotly.graph_objects as go
import plotly.express as px

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=Arg12b1f.PM10.index,
    y=Arg12b1f.PM10,
    name='Arg12b1f',
))

fig.add_trace(go.Scatter(
    x=Arg12b2f.PM10.index,
    y=Arg12b2f.PM10,
    name='Arg12b2f',
))

fig.add_trace(go.Scatter(
    x=Arg12b3f.PM10.index,
    y=Arg12b3f.PM10,
    name='Arg12b3f',
))

# permet de rajouter la visualisation dynamique issue des stockcharts
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.update_layout(title_text='PM10 measures', title_x=0.5)
fig.show()